# Assignment 2A, Part 3: Multifield retrieval

Implement BM25F and the Mixture of Language Models (MLM). Use two fields: title and content.

In [ ]:
QUERY_FILE = "data/queries.txt"  # make sure the query file exists on this location
OUTPUT_FILE = "data/output.txt"  # output the ranking

In [2]:
import numpy as np
import pickle
import math
import pandas as pd

import nltk

from IPython.display import clear_output # Using IPython.display.clear_output to clear the output of a cell.

nltk_stopwords = set(nltk.corpus.stopwords.words('english'))

## Load index

### Note
I have already calculated the indexes, the document length and the P(t|C) in '1_Indexer' so I am only loading that data here.

In [3]:
# TODO: place the indexing related code here. This may be copy-pasted from Part 1.
inv_idx, doc_len, P_tC, avg_dl = [{}, {}], [{}, {}], [{}, {}], [0, 0] # one index for each parameters

In [4]:
# Load indexes, doc_len and P(t|C) for content data
inv_idx[0] = pickle.load(open("data/indexes_content.p", "rb" ))
doc_len[0] = pickle.load(open("data/doc_len_content.p", "rb" ))
P_tC[0] = pickle.load(open("data/P_tC_content.p", "rb" ))
NUM_DOCS = len(doc_len[0])
avg_dl[0] = sum(list(doc_len[0].values()))/NUM_DOCS

In [5]:
inv_idx[1] = pickle.load(open("data/indexes_title.p", "rb" ))
doc_len[1] = pickle.load(open("data/doc_len_title.p", "rb" ))
P_tC[1] = pickle.load(open("data/P_tC_title.p", "rb" ))

avg_dl[1] = sum(list(doc_len[1].values()))/NUM_DOCS

### Load the queries from the file

See the assignment description for the format of the query file [here](https://github.com/kbalog/uis-dat640-fall2019/tree/master/assignments/assignment-2a#queries).

In [6]:
def load_queries(query_file):
    queries = {}
    with open(query_file, "r") as fin:
        for line in fin.readlines():
            qid, query = line.strip().split(" ", 1)
            queries[qid] = query
    return queries

In [7]:
queries = load_queries(QUERY_FILE)

## Retrieval models

In [8]:
# TODO write your scoring code here
def get_BM25f_ranking_for(q_id, query, k1, b, weights):
    scores = {}
    
    q_words = query.lower().replace("-", " ").replace(",", "").split()
    
    for term in list(set(q_words)):
        for i, w_i in enumerate(weights):
            if term in list(inv_idx[i].keys()):
                # If the term is in the inverse index
                _f_td = {}
                
                n = len(inv_idx[i][term])
                idf = math.log((NUM_DOCS - n + 0.5) / (n + 0.5))
                
                for (doc_id, f_td) in inv_idx[i][term].items():                    
                    B_i = 1 - b[i] + b[i]*(doc_len[i][doc_id]/avg_dl[i])
                    
                    _f_td[doc_id] = _f_td.get(doc_id, 0) + (w_i*(f_td/B_i))
                    
                    score_for_term = round(idf*(_f_td[doc_id]/(k1 + _f_td[doc_id])), 3)
                    scores[doc_id] = scores.get(doc_id, 0) + score_for_term
            
    scores = sorted(scores.items(), key=lambda score: score[1], reverse = True)[:100]
    return scores

def get_MLM_ranking_for(q_id, query, lmbda, weights):    
    scores = {}
    q_words = query.lower().replace("-", " ").replace(",", "").split()
    
    for term in list(set(q_words)):
        for i, w_i in enumerate(weights):
            if (term in list(inv_idx[i].keys())):
                # If the term is in the inverse index
                for (doc_id, f_td) in inv_idx[i][term].items():
                    score_for_term = w_i*((1-lmbda[i])*(f_td / doc_len[i][doc_id]) + (lmbda[i]*P_tC[i][term]))
                    
                    scores[doc_id] = scores.get(doc_id, 0) + score_for_term
                
    for doc_id, score in scores.items():
        scores[doc_id] = math.log(scores[doc_id])
    
    scores = sorted(scores.items(), key=lambda score: score[1], reverse = True)[:100]
    
    return scores

### Perform retrieval

**TODO** Generate a ranking for each query and output the results to `OUTPUT_FILE`

See the assignment description for the format of the output file [here](https://github.com/kbalog/uis-dat640-fall2019/tree/master/assignments/assignment-2a#output-file-format).

In [12]:
def BM25f_ranking_data(k1, b, weights):
    BM25f_data = pd.DataFrame(columns=['QueryId', 'DocumentId'])
    
    for q_id, query in queries.items():
        # TODO generate ranking
#         print(q_id, query)
        BM_25f_rankings = get_BM25f_ranking_for(q_id, query, k1 = k1, b = b, weights = weights)

        for score in BM_25f_rankings:
            BM25f_data = BM25f_data.append(pd.DataFrame([[q_id, score[0]]], columns=['QueryId', 'DocumentId']))

    return BM25f_data
    
def MLM_ranking_data(lmbda, weights):
    MLM_data = pd.DataFrame(columns=['QueryId', 'DocumentId'])
    
    for q_id, query in queries.items():
        # TODO generate ranking
#         print(q_id, query)
        MLM_rankings = get_MLM_ranking_for(q_id, query, lmbda = lmbda, weights = weights)

        # TODO write results to file
        for score in MLM_rankings:
            MLM_data = MLM_data.append(pd.DataFrame([[q_id, score[0]]], columns=['QueryId', 'DocumentId']))

    return MLM_data

In [14]:
BM25f_data = BM25f_ranking_data(k1 = 1, b = [0.2, 0.3], weights = [0.9, 0.1])
MLM_data = MLM_ranking_data(lmbda = [0.8,0.8], weights = [0.9, 0.1])

BM25f_data.to_csv("bm25f_multifield.csv", index = False)
MLM_data.to_csv("mlm_multifield.csv", index = False)

In [11]:
# different parameters for the "Parameter Tuning".

# bm25f_params = {
#     'k1': [x for x in np.arange(1.0, 2.1, 0.1)], # k1, calibrating term frequency scaling, can be any value over 0, for the experiment I'm using 1 <= k1 <= 2
#     'b': [(x, y) for x in np.arange(0, 1.1, 0.2) for y in np.arange(0, 1.1, 0.2)], # b, the normalization factors, is a tuple of values in which either of the values can be 0 <= b <= 1
#     'weights': [(x, 1-x) for x in np.arange(0, 1.1, 0.2)], # weights is a tuple of 2 values which is 1 when summed
# }


# mlm_params = {
#     'lmbda': [(x, y) for x in np.arange(0.1, 1.1, 0.1) for y in np.arange(0.1, 1.1, 0.1)], # lambda, the smoothing parameter
#     'weights': [(x, 1-x) for x in np.arange(0.1, 1.0, 0.1)], # weights is a tuple of 2 values which is 1 when summed
# } 

## Evaluation

Report on the evaluation results (using the [Evaluation notebook](1_Evaluation.ipynb)) here.

I tried the brute force approach and defined a range of values for each of the variables, but it takes a very long time. So, I tried some trial and error, but I didn't randomly select the parameters, I used a similar approach as the solution to the 12 marbles puzzle. I did not have enough time to code and let it run to do a comprehensive test of this random theory I just had, so I only did a few iterations manually. The idea is as follows -

All my parameters are in a specific range and also in ascending order. 

First, I calculate accuracy of the 25th and 75th percentile of a parameter and also the accuracy of the median of the set of values for the parameter. Compared to the median, if the accuracy was higher on the 25th percentile as opposed to the 75th percentile I would redefine my set of parameters to all values in the first half of the set, otherwise I would redefine my set of parameters to all values in the second half of the set. Then I would repeat the process until either the index of the 25th percentile is higher or equal to the median or the index of the 75th percentile is lower or equal to the median, in any of these 2 cases I would simply return the median parameter combination. In any other case, I would return the parameter value with the higher accuracy.

If I did this recursively for each parameters, this would significantly reduce the computation time. But for this I had a major assumption that the accuracy would increase or decrease if I simply selected a specific "direction" in each iteration in each parameter.


| **Method** | **Parameter settings** | **Output file** | **P@10** | **MAP** | **MRR** |
| -- | -- | -- | -- | -- | -- |
| BM25F | k1: 11, b: [0.3, 0.25], $w_{title}$: 0.1, $w_{content}$: 0.9 | `data/bm25f_multifield.csv` | 0.174 | 0.066 | 0.282 |
| MLM | Smoothing method: Jelinek-Mercer, smoothing param: lambda = [0.8,0.8], $w_{title}$: 0.1, $w_{content}$: 0.9 | `data/mlm_multifield.csv` | 0.130 | 0.046 | 0.221 |